In [3]:
from keras import backend as K
from keras.models import load_model
import numpy as np

ImportError: cannot import name 'tensor' from 'tensorflow.python.framework' (C:\Users\91781\anaconda\lib\site-packages\tensorflow\python\framework\__init__.py)

In [ ]:
# statistics.py
import numpy as np
import math
from sklearn.metrics import confusion_matrix

goes_classes = ['quiet','A','B','C','M','X']


def flux_to_class(f: float, only_main=False):
    'maps the peak_flux of a flare to one of the following descriptors: \
    *quiet* = 1e-9, *B* >= 1e-7, *C* >= 1e-6, *M* >= 1e-5, and *X* >= 1e-4\
    See also: https://en.wikipedia.org/wiki/Solar_flare#Classification'
    decade = int(min(math.floor(math.log10(f)), -4))
    sub = round(10 ** -decade * f)
    if decade < -4: # avoiding class 10
        decade += sub // 10
        sub = max(sub % 10, 1)
    main_class = goes_classes[decade + 9] if decade >= -8 else 'quiet'
    sub_class = str(sub) if main_class != 'quiet' and only_main != True else ''
    return main_class + sub_class

def class_to_flux(c: str):
    'Inverse of flux_to_class \
    Maps a flare class (e.g. B6, M, X9) to a GOES flux value'
    if c == 'quiet':
        return 1e-9
    decade = goes_classes.index(c[0])-9
    sub = float(c[1:]) if len(c) > 1 else 1
    return round(10 ** decade * sub, 10)


#
#   See https://arxiv.org/pdf/1608.06319.pdf for details about scores and statistics
#

def true_skill_statistic(y_true, y_pred, threshold='M'):
    'Calculates the True Skill Statistic (TSS) on binarized predictions\
    It is not sensitive to the balance of the samples\
    This statistic is often used in weather forecasts (including solar weather)\
    1 = perfect prediction, 0 = random prediction, -1 = worst prediction'
    separator = class_to_flux(threshold)
    y_true = [1 if yt >= separator else 0 for yt in y_true]
    y_pred = [1 if yp >= separator else 0 for yp in y_pred]
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tp / (tp + fn) - fp / (fp + tn)

def heidke_skill_score(y_true, y_pred, threshold='M'):
    'Claculates the Heidke skill score'
    separator = class_to_flux(threshold)
    y_true = [1 if yt >= separator else 0 for yt in y_true]
    y_pred = [1 if yp >= separator else 0 for yp in y_pred]
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return (tp + fn) / len(y_pred) * (tp + fp) / len(y_pred) + (tn + fn) / len(y_pred) * (tn + fp) / len(y_pred)



# keras_generator.py
import keras.utils.data_utils
#from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import load_img
import pandas as pd
import numpy as np
import os
import datetime as dt

class SDOBenchmarkGenerator(keras.utils.data_utils.Sequence):
    'Generates data for keras \
    Inspired by https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly.html'
    def __init__(self, base_path, batch_size=32, dim=(3,224, 224), channels=['magnetogram'], shuffle=True, augment=True, label_func=None, data_format="channels_last"):
        'Initialization'
        self.batch_size = batch_size
        self.base_path = base_path
        self.data_format = data_format
        self.label_func = label_func
        self.dim = dim if len(dim) == 4 else (dim + (len(channels),) if data_format=='channels_last' else (len(channels),) + dim)
        self.channels = channels
        self.time_steps = [0, 7*60, 10*60+30, 11*60+50]
        self.data = self.loadCSV(augment)
        self.shuffle = shuffle
        self.on_epoch_end()

    def loadCSV(self, augment=True):
        data = pd.read_csv(os.path.join(self.base_path, 'meta_data.csv'), sep=",", parse_dates=["start", "end"], index_col="id")

        # augment by doubling the data and flagging them to be flipped horizontally
        data['flip'] = False
        if augment:
            new_data = data.copy()
            new_data.index += '_copy'
            new_data['flip'] = True
            data = pd.concat([data, new_data])
        return data

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.data))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        X = [
            np.empty((self.batch_size, 1)),
            np.empty((self.batch_size, *self.dim))
        ]

        # Generate data
        data = self.data.iloc[indexes]
        X[0] = np.asarray(list(map(self.loadImg, data.index)))
        ind = np.where(data['flip'])
        X[0][ind] = X[0][ind, ::-1, ...]
        X[1] = (data['start'] - pd.Timestamp('2012-01-01 00:00:00')).view('int64')
        X[1] /= (pd.Timestamp('2018-01-01 00:00:00') - pd.Timestamp('2012-01-01 00:00:00')).view('int64')
        y = np.array(data['peak_flux'])
        if self.label_func is not None:
            y = self.label_func(y)
        return X, y

    def loadImg(self, sample_id):
        'Load the images of each timestep as channels'
        ar_nr, p = sample_id.replace('_copy','').split("_", 1)
        path = os.path.join(self.base_path, ar_nr, p)

        slices = np.zeros(self.dim)

        sample_date = dt.datetime.strptime(p[:p.rfind('_')], "%Y_%m_%d_%H_%M_%S")
        time_steps = [sample_date + dt.timedelta(minutes=offset) for offset in self.time_steps]
        for img in [name for name in os.listdir(path) if name.endswith('.jpg')]:
            img_datetime_raw, img_wavelength = os.path.splitext(img)[0].split("__")
            img_datetime = dt.datetime.strptime(img_datetime_raw, "%Y-%m-%dT%H%M%S")

            # calc wavelength and datetime index
            datetime_index = [di[0] for di in enumerate(time_steps) if abs(di[1] - img_datetime) < dt.timedelta(minutes=15)]
            if img_wavelength in self.channels and len(datetime_index) > 0:
                val = np.squeeze(img_to_array(load_img(os.path.join(path, img), grayscale=True)), 2)
                if self.data_format == 'channels_first':
                    slices[datetime_index[0],:,:,self.channels.index(img_wavelength)] = val
                else:
                    slices[self.channels.index(img_wavelength),:,:,datetime_index[0]] = val



        return slices

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

In [3]:
!pip install opendatasets
!pip install pandas

In [5]:
import opendatasets as od
import pandas
 
od.download(
    "https://www.kaggle.com/datasets/fhnw-i4ds/sdobenchmark")


Skipping, found downloaded files in ".\sdobenchmark" (use force=True to force download)


In [2]:
import os
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np

ImportError: cannot import name 'tensor' from 'tensorflow.python.framework' (C:\Users\91781\anaconda\lib\site-packages\tensorflow\python\framework\__init__.py)

In [ ]:
from keras.models import Model
from keras import layers
from keras.layers import Dense, Dropout, Flatten, Input, Lambda, Reshape, Activation
from keras.layers import Conv2D, MaxPooling2D, concatenate, SeparableConv2D, GlobalAveragePooling2D, GlobalMaxPooling2D, TimeDistributed
#from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
#from notebooks.utils.keras_generator import SDOBenchmarkGenerator
import time
import numpy as np

In [ ]:
base_path = 'sdobenchmark/SDOBenchmark_full/'
# Parameters
params = {'dim': (4,224, 224, 3),
          'batch_size': 16,
          'channels': ['magnetogram', '304', '131', '1700'],
          'shuffle': True}

# category encoding quiet, A1..9, B1..9, ... X1..9, X10+
categories = to_categorical(np.arange(7))
params['label_func'] = lambda y: categories[np.floor(np.log10(y)+9).astype(int)]

# Generators
training_generator = SDOBenchmarkGenerator(os.path.join(base_path, 'training'), **params)
validation_generator = SDOBenchmarkGenerator(os.path.join(base_path, 'test'), **params)

category_balance = np.sum(params['label_func'](training_generator.data['peak_flux']), axis=0)
category_weights = np.sum(category_balance) / category_balance
category_weights[np.isinf(category_weights)] = 0
category_weights = dict(zip(range(len(categories)), category_weights))

start_time = time.time()

scales = [
    64, #256,
    64*3, #728,
    1, #8,
    128, #1024,
    256 #2048
]

#try:
images_input = Input(shape=params['dim'], name='images_input')


In [60]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [61]:
import tensorflow as tf
from tensorflow.python.layers.normalization import BatchNormalization
from keras.layers import TimeDistributed
from keras.layers import Reshape, Permute
#input_shape = ( 224, 224, 3)  # Assuming 4 frames of 224x224 images with 3 channels
#reshaped_input = Permute((2, 3, 4, 1))(images_input)  # Permute to (height, width, channels, frames)
#tensor = tf.convert_to_tensor(input_shape)

In [89]:
#reshaped_input = Reshape(target_shape=(4,224, 224, 3))(input_shape)
conv1 = TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"))(images_input)
conv2 = TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"))(conv1)
pool1 = TimeDistributed(MaxPooling2D((2, 2)))(conv2)

conv3 = TimeDistributed(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))(pool1)
conv4 = TimeDistributed(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))(conv3)
#x = TimeDistributed(BatchNormalization(name='block13_sepconv2_bn'))(conv4)
pool2 = TimeDistributed(MaxPooling2D((2, 2)))(conv4)

conv5 = TimeDistributed(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))(pool2)
conv6 = TimeDistributed(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))(conv5)
conv7 = TimeDistributed(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))(conv6)
pool3 = TimeDistributed(MaxPooling2D((2, 2)))(conv7)

conv8 = TimeDistributed(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))(pool3)
conv9 = TimeDistributed(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))(conv8)
conv10 = TimeDistributed(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))(conv9)
pool4 = TimeDistributed(MaxPooling2D((2, 2)))(conv10)

conv11 = TimeDistributed(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))(pool4)
conv12 = TimeDistributed(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))(conv11)
conv13 = TimeDistributed(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))(conv12)
pool5 = TimeDistributed(GlobalMaxPooling2D(name='max_pool'))(conv13 )


In [90]:
import tensorflow as tf
reshaped_pool5 = tf.reshape(pool5, (-1, 7, 7, -1))
feature_map = tf.image.resize(reshaped_pool5, size=(32, 32), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
feature_map = tf.reshape(feature_map, ( 4,32, 32,1))

#feature_map = tf.image.resize(pool5, size=(None, 4,32, 32), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

In [91]:

x = Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1))(feature_map)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(16, kernel_size=(5, 5), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
date_input = Input(shape=(1,), name='date_input')
x = concatenate([x, date_input])
x = Dense(120, activation='relu')(x)
x = Dense(84, activation='relu')(x)
main_output = Dense(len(categories), activation='softmax', name='main_output')(x)


In [92]:
def converted_mae(y_true, y_pred):
    conv_y_true = 10.**(K.cast(K.argmax(y_true, axis=-1), K.floatx())-9.) * 5.
    conv_y_pred = 10.**(K.cast(K.argmax(y_pred, axis=-1), K.floatx())-9.) * 5.
    #K.cast(xyz,K.floatx())
    return K.mean(K.abs(conv_y_pred - conv_y_true), axis=-1)


In [93]:
model = Model(inputs=[images_input, date_input], outputs=[main_output])
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy', converted_mae])

In [94]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 images_input (InputLayer)      [(None, 4, 224, 224  0           []                               
                                , 3)]                                                             
                                                                                                  
 time_distributed_81 (TimeDistr  (None, 4, 224, 224,  1792       ['images_input[0][0]']           
 ibuted)                         64)                                                              
                                                                                                  
 time_distributed_82 (TimeDistr  (None, 4, 224, 224,  36928      ['time_distributed_81[0][0]']    
 ibuted)                         64)                                                        

In [108]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [107]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=10, verbose=1, min_lr=0.0000001) #, epsilon=1e-8
cb_tensorBoard = TensorBoard(log_dir='./logs/n1', histogram_freq=0, write_grads=True, write_graph=False, write_images=True)
cb_modelCheckpoint = ModelCheckpoint('./models/n1_' + os.environ['CUDA_VISIBLE_DEVICES'] + '_{epoch:02d}-{loss:.2f}-{val_loss:.2f}', monitor='val_loss', verbose=1, save_best_only=True)



In [97]:
# Train model on dataset
from tensorflow.keras.utils import img_to_array
history = model.fit_generator(generator=training_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=len(training_generator),
                              validation_steps=len(validation_generator),
                              epochs=1000,
                              class_weight = category_weights,
                              callbacks=[cb_tensorBoard, reduce_lr, cb_modelCheckpoint], #reduce_lr
                              use_multiprocessing=True,
                              max_queue_size=params['batch_size'],
                              workers=params['batch_size'] * 2)

C:\Users\91781\AppData\Local\Temp\ipykernel_11864\4117276879.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=training_generator,


ValueError: could not broadcast input array from shape (256,256) into shape (224,224)

In [106]:
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau

# Define your callbacks
cb_tensorBoard = TensorBoard(log_dir='/path/to/tensorboard/logs')
cb_modelCheckpoint = ModelCheckpoint('/path/to/save/model.h5', save_best_only=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.00001)

# Define your category weights (assuming you have already calculated them)
category_weights = {0: 1, 1: 2, 2: 3}  # Example weights, please replace with actual values

# Train model on dataset
history = model.fit_generator(generator=training_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=len(training_generator),
                              validation_steps=len(validation_generator),
                              epochs=1000,
                              class_weight=category_weights,
                              callbacks=[cb_tensorBoard, reduce_lr, cb_modelCheckpoint],
                              use_multiprocessing=True,
                              max_queue_size=params['batch_size'],
                              workers=params['batch_size'] * 2)


ImportError: cannot import name 'tensor' from 'tensorflow.python.framework' (C:\Users\91781\anaconda\lib\site-packages\tensorflow\python\framework\__init__.py)